In [103]:
import pandas as pd
import numpy as np

In [104]:
#very hard to work with formatting so looks messy
df_ipc=pd.read_excel("Data/Ethiopia - IPC Analysis 2017-2020.xlsx",header=11)

In [105]:
df_ipc.shape

(149, 54)

In [106]:
df_ipc.columns

Index(['Country', 'Level 1 Name', 'Area', 'Analysis Name', 'Date of Analysis',
       'Country Population', '#', '% of total county Pop', 'Area Phase',
       'Analysis Period', '#.1', '%', '#.2', '%.1', '#.3', '%.2', '#.4', '%.3',
       '#.5', '%.4', '#.6', '%.5', '#.7', '% of total county Pop.1',
       'Area Phase.1', 'Analysis Period.1', '#.8', '%.6', '#.9', '%.7', '#.10',
       '%.8', '#.11', '%.9', '#.12', '%.10', '#.13', '%.11', '#.14',
       '% of total county Pop.2', 'Area Phase.2', 'Analysis Period.2', '#.15',
       '%.12', '#.16', '%.13', '#.17', '%.14', '#.18', '%.15', '#.19', '%.16',
       '#.20', '%.17'],
      dtype='object')

In [107]:
df_ipcp=df_ipc[['Level 1 Name','Area',"#","Date of Analysis","Area Phase"]]

In [108]:
df_ipcp=df_ipcp[df_ipcp["Area Phase"].notnull()]

In [109]:
df_ipcp=df_ipcp.rename(columns={"Level 1 Name":"ADM1_EN","Area":"ADM2_EN","#":"pop_CS","Date of Analysis":"date","Area Phase":"CS"})

In [110]:
df_ipcp

,ADM1_EN,ADM2_EN,pop_CS,date,CS
2,Afar,Zone 1 (awsi rasu),391597.0,Aug 2020,3.0
3,Afar,Zone 2 (kilbet rasu),431266.0,Aug 2020,3.0
4,Afar,Zone 3 (gabi rasu),185304.0,Aug 2020,3.0
5,Afar,Zone 4 (fantana rasu),257449.0,Aug 2020,3.0
6,Afar,Zone 5 (hari rasu),274937.0,Aug 2020,2.0
...,...,...,...,...,...
138,Tigray,Mereb lehe,111367.0,Sep 2019,3.0
139,Tigray,RVL - Raya Valley Livelihood Zone,233294.0,Sep 2019,3.0
140,Tigray,Saesie Tsaedamba,134918.0,Sep 2019,3.0
141,Tigray,TSC - Tsirare Catchment Livelihood Zone,125199.0,Sep 2019,3.0


In [111]:
df_ipcp.date=pd.to_datetime(df_sel.date,format="%b %Y")

In [112]:
df_ipcp

,ADM1_EN,ADM2_EN,pop_CS,date,CS
2,Afar,Zone 1 (awsi rasu),391597.0,2020-08-01,3.0
3,Afar,Zone 2 (kilbet rasu),431266.0,2020-08-01,3.0
4,Afar,Zone 3 (gabi rasu),185304.0,2020-08-01,3.0
5,Afar,Zone 4 (fantana rasu),257449.0,2020-08-01,3.0
6,Afar,Zone 5 (hari rasu),274937.0,2020-08-01,2.0
...,...,...,...,...,...
138,Tigray,Mereb lehe,111367.0,2019-09-01,3.0
139,Tigray,RVL - Raya Valley Livelihood Zone,233294.0,2019-09-01,3.0
140,Tigray,Saesie Tsaedamba,134918.0,2019-09-01,3.0
141,Tigray,TSC - Tsirare Catchment Livelihood Zone,125199.0,2019-09-01,3.0


In [113]:
status="CS"
for level in [1, 2, 3, 4, 5]:
    ipc_id = "{}_{}".format(status, level)
    df_ipcp[ipc_id] = np.where(
        df_ipcp[status] == level,
        df_ipcp[f"pop_{status}"],
        (np.where(df_ipcp[status] == 99, np.nan, 0)),
    )

In [114]:
df_ipcp

,ADM1_EN,ADM2_EN,pop_CS,date,CS,CS_1,CS_2,CS_3,CS_4,CS_5
2,Afar,Zone 1 (awsi rasu),391597.0,2020-08-01,3.0,0.0,0.0,391597.0,0.0,0.0
3,Afar,Zone 2 (kilbet rasu),431266.0,2020-08-01,3.0,0.0,0.0,431266.0,0.0,0.0
4,Afar,Zone 3 (gabi rasu),185304.0,2020-08-01,3.0,0.0,0.0,185304.0,0.0,0.0
5,Afar,Zone 4 (fantana rasu),257449.0,2020-08-01,3.0,0.0,0.0,257449.0,0.0,0.0
6,Afar,Zone 5 (hari rasu),274937.0,2020-08-01,2.0,0.0,274937.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
138,Tigray,Mereb lehe,111367.0,2019-09-01,3.0,0.0,0.0,111367.0,0.0,0.0
139,Tigray,RVL - Raya Valley Livelihood Zone,233294.0,2019-09-01,3.0,0.0,0.0,233294.0,0.0,0.0
140,Tigray,Saesie Tsaedamba,134918.0,2019-09-01,3.0,0.0,0.0,134918.0,0.0,0.0
141,Tigray,TSC - Tsirare Catchment Livelihood Zone,125199.0,2019-09-01,3.0,0.0,0.0,125199.0,0.0,0.0


In [115]:
df_ipcp=df_ipcp[["date","ADM1_EN","ADM2_EN","pop_CS"]+[f"CS_{i}" for i in range(1,6)]]

In [116]:
df_ipcp

,date,ADM1_EN,ADM2_EN,pop_CS,CS_1,CS_2,CS_3,CS_4,CS_5
2,2020-08-01,Afar,Zone 1 (awsi rasu),391597.0,0.0,0.0,391597.0,0.0,0.0
3,2020-08-01,Afar,Zone 2 (kilbet rasu),431266.0,0.0,0.0,431266.0,0.0,0.0
4,2020-08-01,Afar,Zone 3 (gabi rasu),185304.0,0.0,0.0,185304.0,0.0,0.0
5,2020-08-01,Afar,Zone 4 (fantana rasu),257449.0,0.0,0.0,257449.0,0.0,0.0
6,2020-08-01,Afar,Zone 5 (hari rasu),274937.0,0.0,274937.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
138,2019-09-01,Tigray,Mereb lehe,111367.0,0.0,0.0,111367.0,0.0,0.0
139,2019-09-01,Tigray,RVL - Raya Valley Livelihood Zone,233294.0,0.0,0.0,233294.0,0.0,0.0
140,2019-09-01,Tigray,Saesie Tsaedamba,134918.0,0.0,0.0,134918.0,0.0,0.0
141,2019-09-01,Tigray,TSC - Tsirare Catchment Livelihood Zone,125199.0,0.0,0.0,125199.0,0.0,0.0


In [121]:
df_ad1=df_ipcp.groupby(["date","ADM1_EN"]).agg(lambda x: np.nan if x.isnull().all() else x.sum()).reset_index()

In [122]:
def get_trigger(row, status, level, perc):
    # range till 6 cause 5 is max level
    cols = [f"{status}_{l}" for l in range(level, 6)]
    if np.isnan(row[f"pop_{status}"]):
        return np.nan
    if row[cols].sum() >= row[f"pop_{status}"] / (100 / perc):
        return 1
    else:
        return 0


def get_trigger_increase(row, level, perc):
    # range till 6 cause 5 is max level
    cols_ml1 = [f"ML1_{l}" for l in range(level, 6)]
    cols_cs = [f"CS_{l}" for l in range(level, 6)]
    if row[["pop_CS", "pop_ML1"]].isnull().values.any():
        return np.nan
    if row[cols_ml1].sum() == 0:
        return 0
    if row[cols_ml1].sum() >= row[cols_cs].sum() * (1 + (perc / 100)):
        return 1
    else:
        return 0

In [123]:
df_ad1['year'] = df_ad1['date'].dt.year
df_ad1['month'] = df_ad1['date'].dt.month

In [124]:
df_ad1["trigger_CS_3_20"]=df_ad1.apply(lambda x: get_trigger(x,"CS",3,20),axis=1)
df_ad1["trigger_CS_4_2"]=df_ad1.apply(lambda x: get_trigger(x,"CS",4,2.5),axis=1)
df_ad1["trigger_CS_4_20"]=df_ad1.apply(lambda x: get_trigger(x,"CS",4,20),axis=1)
df_ad1["trigger_CS_4_1"]=df_ad1.apply(lambda x: get_trigger(x,"CS",4,0.1),axis=1)

In [125]:
for c in ["CS_3","CS_4"]:
    status=c.split("_")[0]
    df_ad1[f"perc_{c}"]=df_ad1[c]/df_ad1[f"pop_{status}"]*100

In [126]:
df_ad1

,date,ADM1_EN,ADM2_EN,pop_CS,CS_1,CS_2,CS_3,CS_4,CS_5,year,month,trigger_CS_3_20,trigger_CS_4_2,trigger_CS_4_20,trigger_CS_4_1,perc_CS_3,perc_CS_4
0,2019-09-01,Afar,AAP - Assale Agropastoral Livelihood ZoneAGA -...,1513546.0,0.0,104012.0,1409534.0,0.0,0.0,2019,9,1,0,0,0,93.127926,0.0
1,2019-09-01,Amhara,ABB - Abay-Beshilo Basin Livelihood ZoneEast b...,3457931.0,0.0,1127792.0,2330139.0,0.0,0.0,2019,9,1,0,0,0,67.385353,0.0
2,2019-09-01,Oromiya,"Afc-Agarfa, Gasera, Ginir & Gololcha Fruit, Co...",11497625.0,0.0,33873.0,11463752.0,0.0,0.0,2019,9,1,0,0,0,99.705391,0.0
3,2019-09-01,SNNPR,AMP Alaba-Mareko Lowland Pepper LZAbm-Alaba-Ba...,5600673.0,0.0,2131900.0,3468773.0,0.0,0.0,2019,9,1,0,0,0,61.934932,0.0
4,2019-09-01,Somali,AFP - Afder Pastoral Livelihood ZoneDAP - Dega...,5158000.0,0.0,0.0,5158000.0,0.0,0.0,2019,9,1,0,0,0,100.000000,0.0
5,2019-09-01,Tigray,AoH - Alaje-Ofla Highland Livelihood ZoneEDM -...,1499302.0,0.0,0.0,1499302.0,0.0,0.0,2019,9,1,0,0,0,100.000000,0.0
6,2020-08-01,Afar,Zone 1 (awsi rasu)Zone 2 (kilbet rasu)Zone 3 (...,1540553.0,0.0,274937.0,1265616.0,0.0,0.0,2020,8,1,0,0,0,82.153357,0.0
7,2020-08-01,Amhara,North shewaNorth welloSouth wello,6084363.0,0.0,4603262.0,1481101.0,0.0,0.0,2020,8,1,0,0,0,24.342745,0.0
8,2020-08-01,Oromia,ArsiBaleBorenaEast harargeEast shewaGujiWest a...,16937642.0,0.0,4477473.0,12460169.0,0.0,0.0,2020,8,1,0,0,0,73.564957,0.0
9,2020-08-01,SNNP,DawuroGamoGedeoGofaHadiyaKembata TembaroSegenS...,7404334.0,0.0,2637195.0,4767139.0,0.0,0.0,2020,8,1,0,0,0,64.383090,0.0


In [127]:
dict_an={}

In [128]:
#Analysis: IPC4+ at 20% (current situation)
df_snCS420 = df_ad1.loc[df_ad1['trigger_CS_4_20']==1]
display(df_snCS420.groupby(['year', 'month'], as_index=False)['ADM1_EN'].agg(lambda x: list(x)))
dict_an["an4_20"]={"df":df_snCS420,"trig_cols":["CS_4"],"desc":"IPC4+ at 20% (current situation)"}

""


In [129]:
#Analysis: IPC3+ at 20% (current situation)
df_snCS420 = df_ad1.loc[df_ad1['trigger_CS_3_20']==1]
display(df_snCS420.groupby(['year', 'month'], as_index=False)['ADM1_EN'].agg(lambda x: list(x)))
dict_an["an3_20"]={"df":df_snCS420,"trig_cols":["CS_3"],"desc":"IPC3+ at 20% (current situation)"}

,year,month,ADM1_EN
0,2019,9,"[Afar, Amhara, Oromiya, SNNPR, Somali, Tigray]"
1,2020,8,"[Afar, Amhara, Oromia, SNNP, Somali]"


In [130]:
#never CS4 value
df_ad1.CS_4.unique()

array([0.])